In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("greninja2006/boujdour")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/boujdour


In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/boujdour/Boujdour 10T.csv', sep=";")
df.head()

,DateTime,zone1,zone2,zone3
0,14/09/2022 17:10,"59,81","14,88","60,77"
1,14/09/2022 17:20,"59,68","15,08","60,52"
2,14/09/2022 17:30,"60,45","15,25","60,63"
3,14/09/2022 17:40,"59,72","15,15","59,29"
4,14/09/2022 17:50,"60,75","15,60","60,43"


In [3]:
for col in df.columns[1:]:
  df[col]=df[col].str.replace(",",".",regex=False)


In [4]:
for col in df.columns[1:]:
  df[col]=df[col].astype(float)

In [5]:
# Ensure 'DateTime' is datetime type
df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True, errors='coerce')

# Set DateTime as index
df = df.set_index('DateTime')

# Sort by datetime just in case
df = df.sort_index()

# Now resampling works
data_hourly = df.resample('1h').sum()
data_hourly_mean = df.resample('1h').mean()
data_daily_mean = data_hourly_mean.resample('1D').mean()


In [6]:
!pip install optuna

In [7]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2

In [9]:
!pip install prophet lightgbm optuna tensorflow -q


In [10]:
# ================================================================
# 📘 Robust Hybrid Feature Engineering Pipeline
# Prophet (Daily) + LSTM (Hourly) + Symbolic & Programmatic Features
# ================================================================

import numpy as np
import pandas as pd
from datetime import datetime

# ================================================================
# 1️⃣ LOAD AND PREPARE DATA
# ================================================================
# Assume you already have:
#  🔹 data → original 10-min resolution dataframe
#  🔹 data_hourly_mean → hourly mean dataframe (aggregated from data)
# Example: data_hourly_mean = data.resample('H').mean()

df_hourly = data_hourly_mean.copy()
df_hourly.index.name = "DateTime"

print(f"Raw hourly data shape: {df_hourly.shape}")
print(df_hourly.head())

# ================================================================
# 2️⃣ FEATURE ENGINEERING UTILITIES
# ================================================================

def add_time_features(df):
    """Add calendar and cyclical time features."""
    df = df.copy()
    df["hour"] = df.index.hour
    df["dayofweek"] = df.index.dayofweek
    df["month"] = df.index.month
    df["is_weekend"] = df["dayofweek"].isin([5,6]).astype(int)
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    return df

def add_lag_and_rolling(df, zones, lags=[1,3,6,12,24], rolls=[3,6,12,24]):
    """Add lag and rolling statistical features."""
    df = df.copy()
    for z in zones:
        for l in lags:
            df[f"{z}_lag_{l}"] = df[z].shift(l)
        for w in rolls:
            df[f"{z}_roll_mean_{w}"] = df[z].rolling(window=w, min_periods=1).mean()
            df[f"{z}_roll_std_{w}"] = df[z].rolling(window=w, min_periods=1).std().fillna(0)
    return df

def add_derivatives(df, zones):
    """Add first/second derivatives and percentage change."""
    df = df.copy()
    for z in zones:
        df[f"{z}_diff_1"] = df[z].diff(1)
        df[f"{z}_diff_2"] = df[z].diff(2)
        df[f"{z}_pct_change_1"] = df[z].pct_change(1).replace([np.inf,-np.inf], np.nan).fillna(0)
    return df

def add_fourier_terms(df, period_hours=24, K=3):
    """Add Fourier seasonal terms."""
    df = df.copy()
    t = np.arange(len(df))
    for k in range(1, K+1):
        df[f"fourier_sin_{k}"] = np.sin(2*np.pi*k*t/period_hours)
        df[f"fourier_cos_{k}"] = np.cos(2*np.pi*k*t/period_hours)
    return df

def add_symbolic_like_features(df, zones):
    """Add interpretable symbolic-like nonlinear feature combinations."""
    df = df.copy()
    for z in zones:
        df[f"{z}_sym_sinlag3_logroll6"] = np.sin(df[f"{z}_lag_3"].fillna(0)) * np.log1p(df[f"{z}_roll_mean_6"].fillna(0))
        df[f"{z}_sym_lag1_over_lag24"] = df[f"{z}_lag_1"] / (df[f"{z}_lag_24"].replace(0, np.nan))
        df[f"{z}_sym_prod_diff1_diff2"] = df[f"{z}_diff_1"].fillna(0) * df[f"{z}_diff_2"].fillna(0)
    return df

# ================================================================
# 3️⃣ APPLY PROGRAMMATIC + SYMBOLIC FEATURE ENGINEERING
# ================================================================
zones = [c for c in df_hourly.columns if c.startswith("zone")]
df = df_hourly.copy()
df = add_time_features(df)
df = add_lag_and_rolling(df, zones)
df = add_derivatives(df, zones)
df = add_fourier_terms(df, period_hours=24, K=2)
df = add_symbolic_like_features(df, zones)

print(f"✅ After feature engineering: {df.shape[1]} columns")

# ================================================================
# 4️⃣ PROPHET-DERIVED DAILY FEATURES (TREND + WEEKLY + YEARLY)
# ================================================================
try:
    from prophet import Prophet
    prophet_available = True
except:
    try:
        from fbprophet import Prophet
        prophet_available = True
    except:
        prophet_available = False

if prophet_available:
    print("🧭 Prophet detected — extracting daily components...")
    daily = df_hourly.sum(axis=1).resample("D").mean().reset_index()
    daily.columns = ["ds", "y"]

    m = Prophet(daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=True)
    m.fit(daily)
    forecast = m.predict(m.make_future_dataframe(periods=0, freq="D"))
    comp = forecast[["ds", "trend", "weekly", "yearly", "yhat"]].set_index("ds")
    comp["residual"] = daily.set_index("ds")["y"] - comp["yhat"]

    # Upsample to hourly and align with df
    comp_hourly = comp.reindex(pd.date_range(comp.index.min(), comp.index.max(), freq="H")).ffill()
    comp_hourly = comp_hourly.reindex(df.index, method="ffill")
    for col in comp_hourly.columns:
        df[f"prophet_{col}"] = comp_hourly[col].values
else:
    print("⚙️ Prophet not available — using STL decomposition fallback.")
    from statsmodels.tsa.seasonal import STL
    daily = df_hourly.sum(axis=1).resample("D").mean()
    stl = STL(daily.interpolate(), period=7)
    res = stl.fit()
    comp = pd.DataFrame({
        "trend": res.trend,
        "seasonal": res.seasonal,
        "resid": res.resid
    })
    comp_hourly = comp.reindex(pd.date_range(comp.index.min(), comp.index.max(), freq="H")).ffill()
    comp_hourly = comp_hourly.reindex(df.index, method="ffill")
    df["prophet_trend"] = comp_hourly["trend"].values
    df["prophet_weekly"] = comp_hourly["seasonal"].values
    df["prophet_residual"] = comp_hourly["resid"].values

# ================================================================
# 5️⃣ LSTM-DERIVED TEMPORAL EMBEDDINGS (OPTIONAL)
# ================================================================
try:
    import tensorflow as tf
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import LSTM, Dense, Input
    from tensorflow.keras.callbacks import EarlyStopping
    tf_available = True
except:
    tf_available = False

if tf_available:
    print("🔶 TensorFlow available — training LSTM encoder...")
    feature_cols = [c for c in df.columns if not c.startswith("zone")] + [f"{z}_lag_1" for z in zones]
    feature_cols = [c for c in feature_cols if c in df.columns]
    df_train = df[feature_cols].fillna(method="ffill").fillna(0)
    seq_len = 24  # one-day lookback window

    X, y = [], []
    total = df_hourly.sum(axis=1)
    for i in range(len(df_train)-seq_len):
        X.append(df_train.iloc[i:i+seq_len].values)
        y.append(total.iloc[i+seq_len])
    X, y = np.array(X), np.array(y)

    if len(X) > 0:
        inp = Input(shape=(X.shape[1], X.shape[2]))
        lstm_layer = LSTM(32, return_sequences=False, name="encoder_lstm")(inp)
        out = Dense(1, activation="linear")(lstm_layer)
        model = Model(inputs=inp, outputs=out)
        model.compile(optimizer="adam", loss="mse")
        es = EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)
        model.fit(X, y, epochs=30, batch_size=32, callbacks=[es], verbose=0)

        encoder = Model(inputs=inp, outputs=model.get_layer("encoder_lstm").output)
        embeddings = encoder.predict(X, verbose=0)
        emb_df = pd.DataFrame(embeddings, index=df.index[seq_len:seq_len+len(embeddings)])
        for i_col in range(emb_df.shape[1]):
            df[f"lstm_emb_{i_col}"] = np.nan
            df.loc[emb_df.index, f"lstm_emb_{i_col}"] = emb_df.iloc[:, i_col].values
    else:
        print("Not enough samples for LSTM embedding.")
else:
    print("❌ TensorFlow not available — skipping LSTM embedding features.")

# ================================================================
# 6️⃣ SAVE & DISPLAY FINAL FEATURE DATASET
# ================================================================
print(f"\n✅ Final engineered DataFrame shape: {df.shape}")
print(f"✅ Total columns: {len(df.columns)}")
print(df.head())

df.to_csv("final_engineered_df.csv")
print("💾 Saved as final_engineered_df.csv")


Raw hourly data shape: (14816, 3)
                         zone1      zone2      zone3
DateTime                                            
2022-09-14 17:00:00  60.082000  15.192000  60.328000
2022-09-14 18:00:00  64.758333  16.280000  58.718333
2022-09-14 19:00:00  66.251667  17.761667  54.316667
2022-09-14 20:00:00  79.946667  24.691667  64.728333
2022-09-14 21:00:00  86.553333  25.910000  70.788333
✅ After feature engineering: 70 columns


/tmp/ipykernel_47/2474188632.py:56: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f"{z}_pct_change_1"] = df[z].pct_change(1).replace([np.inf,-np.inf], np.nan).fillna(0)
/tmp/ipykernel_47/2474188632.py:56: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df[f"{z}_pct_change_1"] = df[z].pct_change(1).replace([np.inf,-np.inf], np.nan).fillna(0)
/tmp/ipykernel_47/2474188632.py:56: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to

🧭 Prophet detected — extracting daily components...


17:42:36 - cmdstanpy - INFO - Chain [1] start processing
17:42:36 - cmdstanpy - INFO - Chain [1] done processing
/tmp/ipykernel_47/2474188632.py:115: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  comp_hourly = comp.reindex(pd.date_range(comp.index.min(), comp.index.max(), freq="H")).ffill()
2026-01-11 17:42:38.434596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768153358.647323      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768153358.700844      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔶 TensorFlow available — training LSTM encoder...


/tmp/ipykernel_47/2474188632.py:152: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_train = df[feature_cols].fillna(method="ffill").fillna(0)
I0000 00:00:1768153374.216171      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1768153377.280773     149 cuda_dnn.cc:529] Loaded cuDNN version 90300



✅ Final engineered DataFrame shape: (14816, 107)
✅ Total columns: 107
                         zone1      zone2      zone3  hour  dayofweek  month  \
DateTime                                                                       
2022-09-14 17:00:00  60.082000  15.192000  60.328000    17          2      9   
2022-09-14 18:00:00  64.758333  16.280000  58.718333    18          2      9   
2022-09-14 19:00:00  66.251667  17.761667  54.316667    19          2      9   
2022-09-14 20:00:00  79.946667  24.691667  64.728333    20          2      9   
2022-09-14 21:00:00  86.553333  25.910000  70.788333    21          2      9   

                     is_weekend  hour_sin      hour_cos  zone1_lag_1  ...  \
DateTime                                                              ...   
2022-09-14 17:00:00           0 -0.965926 -2.588190e-01          NaN  ...   
2022-09-14 18:00:00           0 -1.000000 -1.836970e-16    60.082000  ...   
2022-09-14 19:00:00           0 -0.965926  2.588190e-01    6

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


💾 Saved as final_engineered_df.csv


In [ ]:
# ================================================================
# Robust Hybrid Feature Engineering + Visualization Pipeline
# Prophet (Daily) + LSTM (Hourly) + Symbolic & Programmatic Features
# ================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

plt.style.use("seaborn-v0_8-whitegrid")

# Assume you already have:
#  🔹 data → original 10-min resolution dataframe
#  🔹 data_hourly_mean → hourly mean dataframe
# Example: data_hourly_mean = data.resample('H').mean()

df_hourly = data_hourly_mean.copy()
df_hourly.index.name = "DateTime"

print(f"Raw hourly data shape: {df_hourly.shape}")

# --- Plot 1: Original vs Hourly Aggregated ---
plt.figure(figsize=(10, 4))
for c in df_hourly.columns[:2]:
    plt.plot(df_hourly.index[:500], df_hourly[c].iloc[:500], label=c)
plt.title("Sample Hourly Power Demand (First 500 Samples)")
plt.xlabel("Time"); plt.ylabel("Power (kW)")
plt.legend(); plt.tight_layout()
plt.savefig("plot_hourly_timeseries.png", dpi=300)
plt.close()

def add_time_features(df):
    df = df.copy()
    df["hour"] = df.index.hour
    df["dayofweek"] = df.index.dayofweek
    df["month"] = df.index.month
    df["is_weekend"] = df["dayofweek"].isin([5,6]).astype(int)
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    return df

def add_lag_and_rolling(df, zones, lags=[1,3,6,12,24], rolls=[3,6,12,24]):
    df = df.copy()
    for z in zones:
        for l in lags:
            df[f"{z}_lag_{l}"] = df[z].shift(l)
        for w in rolls:
            df[f"{z}_roll_mean_{w}"] = df[z].rolling(window=w, min_periods=1).mean()
            df[f"{z}_roll_std_{w}"] = df[z].rolling(window=w, min_periods=1).std().fillna(0)
    return df

def add_derivatives(df, zones):
    df = df.copy()
    for z in zones:
        df[f"{z}_diff_1"] = df[z].diff(1)
        df[f"{z}_diff_2"] = df[z].diff(2)
        df[f"{z}_pct_change_1"] = df[z].pct_change(1).replace([np.inf,-np.inf], np.nan).fillna(0)
    return df

def add_fourier_terms(df, period_hours=24, K=3):
    df = df.copy()
    t = np.arange(len(df))
    for k in range(1, K+1):
        df[f"fourier_sin_{k}"] = np.sin(2*np.pi*k*t/period_hours)
        df[f"fourier_cos_{k}"] = np.cos(2*np.pi*k*t/period_hours)
    return df

def add_symbolic_like_features(df, zones):
    df = df.copy()
    for z in zones:
        df[f"{z}_sym_sinlag3_logroll6"] = np.sin(df[f"{z}_lag_3"].fillna(0)) * np.log1p(df[f"{z}_roll_mean_6"].fillna(0))
        df[f"{z}_sym_lag1_over_lag24"] = df[f"{z}_lag_1"] / (df[f"{z}_lag_24"].replace(0, np.nan))
        df[f"{z}_sym_prod_diff1_diff2"] = df[f"{z}_diff_1"].fillna(0) * df[f"{z}_diff_2"].fillna(0)
    return df


zones = [c for c in df_hourly.columns if c.startswith("zone")]
df = df_hourly.copy()
df = add_time_features(df)
df = add_lag_and_rolling(df, zones)
df = add_derivatives(df, zones)
df = add_fourier_terms(df, period_hours=24, K=2)
df = add_symbolic_like_features(df, zones)

print(f"✅ After feature engineering: {df.shape[1]} columns")

# --- Plot 2: Rolling Mean Example ---
plt.figure(figsize=(10, 4))
zone_ex = zones[0]
plt.plot(df_hourly.index[:200], df_hourly[zone_ex].iloc[:200], label="Original")
plt.plot(df_hourly.index[:200], df[f"{zone_ex}_roll_mean_6"].iloc[:200], label="Rolling Mean (6)")
plt.title(f"Rolling Mean Feature Example - {zone_ex}")
plt.xlabel("Time"); plt.ylabel("Power (kW)")
plt.legend(); plt.tight_layout()
plt.savefig("plot_rolling_mean.png", dpi=300)
plt.close()

# --- Plot 3: Fourier Components ---
plt.figure(figsize=(6, 3))
plt.plot(df.index[:200], df["fourier_sin_1"].iloc[:200], label="sin(1)")
plt.plot(df.index[:200], df["fourier_cos_1"].iloc[:200], label="cos(1)")
plt.title("Fourier Seasonal Components")
plt.legend(); plt.tight_layout()
plt.savefig("plot_fourier_terms.png", dpi=300)
plt.close()

# --- Plot 4: Symbolic Feature Relationship ---
plt.figure(figsize=(6, 4))
sns.scatterplot(x=df[f"{zone_ex}_sym_sinlag3_logroll6"], y=df[f"{zone_ex}_sym_prod_diff1_diff2"], s=10)
plt.title("Symbolic Feature Interaction")
plt.xlabel("sinlag3_logroll6"); plt.ylabel("prod_diff1_diff2")
plt.tight_layout(); plt.savefig("plot_symbolic_scatter.png", dpi=300); plt.close()

try:
    from prophet import Prophet
    prophet_available = True
except:
    try:
        from fbprophet import Prophet
        prophet_available = True
    except:
        prophet_available = False

if prophet_available:
    daily = df_hourly.sum(axis=1).resample("D").mean().reset_index()
    daily.columns = ["ds", "y"]

    m = Prophet(daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=True)
    m.fit(daily)
    forecast = m.predict(m.make_future_dataframe(periods=0, freq="D"))
    comp = forecast[["ds", "trend", "weekly", "yearly", "yhat"]].set_index("ds")
    comp["residual"] = daily.set_index("ds")["y"] - comp["yhat"]

    comp_hourly = comp.reindex(pd.date_range(comp.index.min(), comp.index.max(), freq="H")).ffill()
    comp_hourly = comp_hourly.reindex(df.index, method="ffill")
    for col in comp_hourly.columns:
        df[f"prophet_{col}"] = comp_hourly[col].values

    # --- Plot 5: Prophet Components ---
    fig, axs = plt.subplots(3, 1, figsize=(7, 6))
    axs[0].plot(comp.index, comp["trend"]); axs[0].set_title("Prophet Trend Component")
    axs[1].plot(comp.index, comp["weekly"]); axs[1].set_title("Prophet Weekly Component")
    axs[2].plot(comp.index, comp["residual"]); axs[2].set_title("Prophet Residuals")
    plt.tight_layout(); plt.savefig("plot_prophet_components.png", dpi=300); plt.close()


try:
    import tensorflow as tf
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import LSTM, Dense, Input
    from tensorflow.keras.callbacks import EarlyStopping
    tf_available = True
except:
    tf_available = False

if tf_available:
    feature_cols = [c for c in df.columns if not c.startswith("zone")] + [f"{z}_lag_1" for z in zones]
    feature_cols = [c for c in feature_cols if c in df.columns]
    df_train = df[feature_cols].fillna(method="ffill").fillna(0)
    seq_len = 24

    X, y = [], []
    total = df_hourly.sum(axis=1)
    for i in range(len(df_train)-seq_len):
        X.append(df_train.iloc[i:i+seq_len].values)
        y.append(total.iloc[i+seq_len])
    X, y = np.array(X), np.array(y)

    if len(X) > 0:
        inp = Input(shape=(X.shape[1], X.shape[2]))
        lstm_layer = LSTM(32, return_sequences=False, name="encoder_lstm")(inp)
        out = Dense(1, activation="linear")(lstm_layer)
        model = Model(inputs=inp, outputs=out)
        model.compile(optimizer="adam", loss="mse")
        es = EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)
        model.fit(X, y, epochs=20, batch_size=32, callbacks=[es], verbose=0)

        encoder = Model(inputs=inp, outputs=model.get_layer("encoder_lstm").output)
        embeddings = encoder.predict(X, verbose=0)

        emb_df = pd.DataFrame(embeddings)
        pca = PCA(n_components=2).fit_transform(embeddings)
        plt.figure(figsize=(6, 4))
        plt.scatter(pca[:, 0], pca[:, 1], s=8, c="royalblue", alpha=0.6)
        plt.title("LSTM Embedding Projection (PCA)")
        plt.tight_layout(); plt.savefig("plot_lstm_embeddings.png", dpi=300); plt.close()


corr = df.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(corr.iloc[:20, :20], cmap="coolwarm", center=0)
plt.title("Feature Correlation Matrix (Sample 20×20)")
plt.tight_layout(); plt.savefig("plot_feature_correlation.png", dpi=300); plt.close()

df.to_csv("final_engineered_df.csv")
print(f"✅ Final engineered dataset saved: {df.shape}")


In [ ]:
# ==============================================================
# Optimized Residual Hybrid: Prophet (daily) + LSTM (hourly residuals)
#    + Optuna Hyperparameter Tuning + 3 Robust Validations + Plots
# ==============================================================

import pandas as pd
import numpy as np
import warnings
import optuna
import tensorflow as tf
import shap
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

warnings.filterwarnings("ignore")
tf.keras.mixed_precision.set_global_policy('mixed_float16')
sns.set_style('whitegrid')

# -------------------- Reproducibility --------------------
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# -------------------- Load Data --------------------
df_hourly = data_hourly_mean.copy()
df_daily = data_daily_mean.copy()
df_hourly.index = pd.to_datetime(df_hourly.index)
df_daily.index = pd.to_datetime(df_daily.index)

# -------------------- Prophet Daily Forecast --------------------
prophet_daily_predictions = pd.DataFrame(index=df_daily.index)
residuals_hourly = pd.DataFrame(index=df_hourly.index)

for zone in df_daily.columns:
    print(f"🔹 Training Prophet (daily) for {zone}")
    prophet_df = df_daily[[zone]].reset_index()
    prophet_df.columns = ["ds", "y"]
    prophet_df.dropna(inplace=True)
    
    m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
    m.fit(prophet_df)
    
    future_daily = pd.DataFrame({"ds": pd.date_range(df_daily.index.min(), df_daily.index.max(), freq='D')})
    forecast_daily = m.predict(future_daily)
    prophet_daily_predictions[zone] = forecast_daily.set_index('ds')['yhat']
    
    # Upsample to hourly
    prophet_hourly = prophet_daily_predictions[zone].reindex(df_hourly.index, method='ffill')
    residuals_hourly[zone] = df_hourly[zone] - prophet_hourly

# -------------------- Feature Engineering --------------------
def create_features(series, index, n_lags=8, window=6):
    df_feat = pd.DataFrame(index=index)
    for l in range(1, n_lags+1):
        df_feat[f'lag_{l}'] = series.shift(l)
    df_feat[f'roll_mean_{window}'] = series.rolling(window).mean()
    df_feat[f'roll_std_{window}'] = series.rolling(window).std()
    df_feat['hour'] = index.hour
    df_feat['dow'] = index.dayofweek
    df_feat['month'] = index.month
    return df_feat.dropna()

# -------------------- LSTM Model --------------------
def build_lstm(input_shape, p):
    model = Sequential([
        LSTM(p['units1'], activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(p['dropout']),
        LSTM(p['units2'], activation='relu'),
        Dense(1, dtype='float32')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=p['lr']), loss='mse')
    return model

# -------------------- Optuna Objective --------------------
def optuna_objective(trial, X_train, y_train, X_val, y_val):
    p = {
        "units1": trial.suggest_int("units1", 64, 128),
        "units2": trial.suggest_int("units2", 32, 64),
        "dropout": trial.suggest_float("dropout", 0.1, 0.3),
        "lr": trial.suggest_float("lr", 1e-4, 5e-3, log=True),
        "batch": trial.suggest_categorical("batch", [16, 32])
    }
    X_train_3d, X_val_3d = X_train[..., None], X_val[..., None]
    model = build_lstm((X_train.shape[1], 1), p)
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train_3d, y_train, validation_data=(X_val_3d, y_val),
              epochs=50, batch_size=p['batch'], verbose=0, callbacks=[es])
    pred = model.predict(X_val_3d, verbose=0).flatten()
    return np.sqrt(mean_squared_error(y_val, pred))

# -------------------- Robust Validations --------------------
def conformal_intervals(y_true, y_pred, alpha=0.05):
    res = np.abs(y_true - y_pred)
    q = np.quantile(res, 1-alpha)
    lower, upper = y_pred - q, y_pred + q
    coverage = np.mean((y_true >= lower) & (y_true <= upper))
    width = np.mean(upper - lower)
    return coverage, width, q, lower, upper

def shap_stability(model, X, feature_names, folds=3, sample=50):
    tscv = TimeSeriesSplit(folds)
    cors = []
    for train_idx, test_idx in tscv.split(X):
        X_bg = X[train_idx][:sample]
        X_test = X[test_idx][:sample]
        predict_fn = lambda x: model.predict(x[...,None], verbose=0).flatten()
        explainer = shap.KernelExplainer(predict_fn, X_bg)
        vals = explainer.shap_values(X_test)
        shap_mean = np.mean(np.abs(vals), axis=0)
        s = pd.Series(shap_mean, index=feature_names)
        cors.append(s)
    corr_vals = [cors[i].corr(cors[i+1], method='spearman') for i in range(len(cors)-1)]
    return np.mean(corr_vals)

def adversarial_validation(X_train, X_val):
    y = np.concatenate([np.zeros(len(X_train)), np.ones(len(X_val))])
    X = np.vstack([X_train, X_val])
    clf = GradientBoostingClassifier()
    clf.fit(X, y)
    preds = clf.predict_proba(X)[:,1]
    auc = roc_auc_score(y, preds)
    return auc

# -------------------- Training & Evaluation --------------------
results = []
train_ratio = 0.8

for zone in df_hourly.columns:
    print(f"\n⚙️ Zone: {zone}")
    feat = create_features(residuals_hourly[zone], df_hourly.index)
    X, y = feat.values, residuals_hourly[zone].loc[feat.index].values
    idx = feat.index
    split = int(len(X)*train_ratio)
    X_train, X_val = X[:split], X[split:]
    y_train, y_val = y[:split], y[split:]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    
    # ---- Optuna search ----
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda tr: optuna_objective(tr, X_train, y_train, X_val, y_val),
                   n_trials=12, show_progress_bar=True)
    best_p = study.best_params
    print("✅ Best Hyperparameters:", best_p)
    
    # ---- Train final LSTM ----
    model = build_lstm((X_train.shape[1],1), best_p)
    es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    model.fit(X_train[...,None], y_train, epochs=50, batch_size=best_p['batch'], verbose=0, callbacks=[es])
    
    # ---- Predictions & Hybrid Forecast ----
    y_val_pred = model.predict(X_val[...,None], verbose=0).flatten()
    prophet_val = prophet_daily_predictions[zone].reindex(idx, method='ffill')[split:]
    hybrid_val = prophet_val.values + y_val_pred
    y_true_val = df_hourly[zone].loc[idx[split:]].values
    
    # ---- Metrics ----
    rmse = np.sqrt(mean_squared_error(y_true_val, hybrid_val))
    r2 = r2_score(y_true_val, hybrid_val)
    print(f"RMSE={rmse:.3f}, R²={r2:.3f}")
    
    # ---- Robust Validations ----
    cov, width, q, lower, upper = conformal_intervals(y_true_val, hybrid_val)
    shap_idx = shap_stability(model, X_train, feat.columns)
    adv_auc = adversarial_validation(X_train, X_val)
    
    results.append({
        "Zone": zone, "RMSE": rmse, "R2": r2,
        "CPI_Coverage": cov, "CPI_Width": width,
        "SHAP_Stability": shap_idx, "Adversarial_AUC": adv_auc
    })
    
    # ---- Plots ----
    plt.figure(figsize=(12,5))
    plt.plot(idx[split:], y_true_val, label='Actual', color='black')
    plt.plot(idx[split:], prophet_val, label='Prophet', alpha=0.6)
    plt.plot(idx[split:], hybrid_val, label='Hybrid', color='orange')
    plt.fill_between(idx[split:], lower, upper, color='gray', alpha=0.2, label='CPI band')
    plt.title(f"{zone}: Actual vs Predicted (Hybrid)")
    plt.legend(); plt.show()
    
    plt.figure(figsize=(10,4))
    sns.histplot(y_true_val - hybrid_val, bins=30, kde=True, color='red')
    plt.title(f"{zone}: Residual Distribution")
    plt.show()
    
    plt.figure(figsize=(8,4))
    sns.kdeplot(np.abs(y_true_val - hybrid_val), fill=True)
    plt.title(f"{zone}: Error Density")
    plt.show()

# -------------------- Summary --------------------
res_df = pd.DataFrame(results)
print("\n✅ Final Results:")
print(res_df.round(4))

plt.figure(figsize=(10,5))
sns.barplot(data=res_df.melt(id_vars="Zone", value_vars=["R2","CPI_Coverage","SHAP_Stability"]),
            x="Zone", y="value", hue="variable")
plt.title("Validation Summary Across Zones")
plt.tight_layout()
plt.show()